### Set up functions and routes

In [1]:
from datetime import datetime
from zoneinfo import ZoneInfo


def get_time(timezone: str) -> str:
    """Finds the current time in a specific timezone.

    :param timezone: The timezone to find the current time in, should
        be a valid timezone from the IANA Time Zone Database like
        "America/New_York" or "Europe/London".
    :type timezone: str
    :return: The current time in the specified timezone."""
    now = datetime.now(ZoneInfo(timezone))
    print(f"Invoked `get_time` function with timezone: `{timezone}`")
    return now.strftime("%H:%M")


def get_news(category: str, country: str) -> str:
    """Useful to get the news in a specific country"""
    print(
        f"Invoked: `get_news` function with category: `{category}` "
        f"and country: `{country}`"
    )
    return "Results from dummy news API"

In [2]:
get_time("America/New_York")

Invoked `get_time` function with timezone: `America/New_York`


'12:59'

Define the LLM

In [3]:
from semantic_router.llms import OpenAILLM
llm = OpenAILLM()

c:\Users\Siraj\Documents\Personal\Work\Aurelio\Virtual Environments\semantic_router_3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Now generate a dynamic routing config for each function

In [4]:
from semantic_router import Route

functions = [get_time, get_news]

routes = []

for function in functions:
    route = Route.from_dynamic_route(llm=llm, entity=function)
    routes.append(route)

2024-05-07 20:59:12 INFO semantic_router.utils.logger Generating dynamic route...
2024-05-07 20:59:15 INFO semantic_router.utils.logger Generated route config:
{
            "name": "get_time",
            "utterances": [
                "What time is it in New York?",
                "Can you tell me the current time in London?",
                "Get the time in Tokyo.",
                "What's the time in Sydney?",
                "Tell me the current time in Los Angeles."
            ]
        }
2024-05-07 20:59:15 INFO semantic_router.utils.logger Generating dynamic route...
2024-05-07 20:59:17 INFO semantic_router.utils.logger Generated route config:
{
            "name": "get_news",
            "utterances": [
                "What's the latest news in the United States?",
                "Tell me about the top headlines in Canada",
                "Get me the news for technology in Japan",
                "Show me the news for sports in Australia",
                "Any updates o

In [5]:
# You can manually add or remove routes

get_weather_route = Route(
    name="get_weather",
    utterances=[
        "what is the weather in SF",
        "what is the current temperature in London?",
        "tomorrow's weather in Paris?",
    ],
    function_schemas=None,
)
routes.append(get_weather_route)

Add routes to the layer config

In [6]:
from semantic_router.layer import LayerConfig

layer_config = LayerConfig(routes=routes)
layer_config.to_dict()

2024-05-07 20:59:18 INFO semantic_router.utils.logger Using default openai encoder: text-embedding-ada-002


{'encoder_type': 'openai',
 'encoder_name': 'text-embedding-ada-002',
 'routes': [{'name': 'get_time',
   'utterances': ['What time is it in New York?',
    'Can you tell me the current time in London?',
    'Get the time in Tokyo.',
    "What's the time in Sydney?",
    'Tell me the current time in Los Angeles.'],
   'description': None,
   'function_schema': {'name': 'get_time',
    'description': 'Finds the current time in a specific timezone.\n\n:param timezone: The timezone to find the current time in, should\n    be a valid timezone from the IANA Time Zone Database like\n    "America/New_York" or "Europe/London".\n:type timezone: str\n:return: The current time in the specified timezone.',
    'signature': '(timezone: str) -> str',
    'output': "<class 'str'>"},
   'llm': {'module': 'semantic_router.llms.openai',
    'class': 'OpenAILLM',
    'model': 'gpt-3.5-turbo'},
   'score_threshold': None},
  {'name': 'get_news',
   'utterances': ["What's the latest news in the United Stat

In [7]:
# Get a route by name
layer_config.get("get_time")

Route(name='get_time', utterances=['What time is it in New York?', 'Can you tell me the current time in London?', 'Get the time in Tokyo.', "What's the time in Sydney?", 'Tell me the current time in Los Angeles.'], description=None, function_schema={'name': 'get_time', 'description': 'Finds the current time in a specific timezone.\n\n:param timezone: The timezone to find the current time in, should\n    be a valid timezone from the IANA Time Zone Database like\n    "America/New_York" or "Europe/London".\n:type timezone: str\n:return: The current time in the specified timezone.', 'signature': '(timezone: str) -> str', 'output': "<class 'str'>"}, llm=OpenAILLM(name='gpt-3.5-turbo', client=<openai.OpenAI object at 0x0000025A67092E90>, temperature=0.01, max_tokens=200), score_threshold=None)

In [8]:
# Remove a route by name
layer_config.remove("get_weather")
layer_config.to_dict()

2024-05-07 20:59:18 INFO semantic_router.utils.logger Removed route `get_weather`


{'encoder_type': 'openai',
 'encoder_name': 'text-embedding-ada-002',
 'routes': [{'name': 'get_time',
   'utterances': ['What time is it in New York?',
    'Can you tell me the current time in London?',
    'Get the time in Tokyo.',
    "What's the time in Sydney?",
    'Tell me the current time in Los Angeles.'],
   'description': None,
   'function_schema': {'name': 'get_time',
    'description': 'Finds the current time in a specific timezone.\n\n:param timezone: The timezone to find the current time in, should\n    be a valid timezone from the IANA Time Zone Database like\n    "America/New_York" or "Europe/London".\n:type timezone: str\n:return: The current time in the specified timezone.',
    'signature': '(timezone: str) -> str',
    'output': "<class 'str'>"},
   'llm': {'module': 'semantic_router.llms.openai',
    'class': 'OpenAILLM',
    'model': 'gpt-3.5-turbo'},
   'score_threshold': None},
  {'name': 'get_news',
   'utterances': ["What's the latest news in the United Stat

Save config to a file (.json or .yaml)

In [9]:
layer_config.to_file("output/layer_config.json")

2024-05-07 20:59:18 INFO semantic_router.utils.logger Saving route config to output/layer_config.json


### Define routing layer

Load config from local file

In [10]:
from semantic_router.layer import LayerConfig

layer_config = LayerConfig.from_file("output/layer_config.json")

2024-05-07 20:59:18 INFO semantic_router.utils.logger Loading route config from output/layer_config.json


Initialize routing layer

In [11]:
import os
from getpass import getpass
from semantic_router import RouteLayer

# https://dashboard.cohere.com/
os.environ["COHERE_API_KEY"] = os.getenv("COHERE_API_KEY") or getpass(
    "Enter Cohere API Key: "
)

layer = RouteLayer.from_config(config=layer_config)

2024-05-07 20:59:18 INFO semantic_router.utils.logger local


Do a function call with functions as tool

In [12]:
layer("What is the time in Stockholm?")

2024-05-07 20:59:19 INFO semantic_router.utils.logger Extracting function input...
2024-05-07 20:59:20 INFO semantic_router.utils.logger LLM output: {
	"timezone": "Europe/Stockholm"
}
2024-05-07 20:59:20 INFO semantic_router.utils.logger Function inputs: {'timezone': 'Europe/Stockholm'}


RouteChoice(name='get_time', function_call={'timezone': 'Europe/Stockholm'}, similarity_score=None)

Define function execution method

In [17]:
from semantic_router.schema import RouteChoice
from semantic_router.schema import Message

llm = OpenAILLM()

def route_and_execute(query, functions, layer):
    route_choice: RouteChoice = layer(query)

    for function in functions:
        if function.__name__ == route_choice.name:
            if route_choice.function_call:
                return function(**route_choice.function_call)

    # If no function is found, use the LLM for general queries
    msgs = [Message(role="user", content=query)]
    return llm(msgs)


queries = [
    "What is the time in Stockholm?",
    "What are the tech news in the US?",
    "The capital of France?",
]

for query in queries:
    print(f"Query: {query}")
    print(route_and_execute(query, functions, layer))

Query: What is the time in Stockholm?


2024-05-07 21:13:23 INFO semantic_router.utils.logger Extracting function input...
2024-05-07 21:13:24 INFO semantic_router.utils.logger LLM output: {
	"timezone": "Europe/Stockholm"
}
2024-05-07 21:13:24 INFO semantic_router.utils.logger Function inputs: {'timezone': 'Europe/Stockholm'}


Invoked `get_time` function with timezone: `Europe/Stockholm`
19:13
Query: What are the tech news in the US?


2024-05-07 21:13:24 INFO semantic_router.utils.logger Extracting function input...
2024-05-07 21:13:25 INFO semantic_router.utils.logger LLM output: {
	"category": "tech",
	"country": "US"
}
2024-05-07 21:13:25 INFO semantic_router.utils.logger Function inputs: {'category': 'tech', 'country': 'US'}


Invoked: `get_news` function with category: `tech` and country: `US`
Results from dummy news API
Query: The capital of France?
##################################################
query
The capital of France?
##################################################
Paris
